In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun
import numpy as np
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [5]:
numbers = np.arange(1, 50).tolist()
rdd_numbers = sc.parallelize(numbers)
rdd_numbers.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

Find the sum, average, maximum, minimum, and count

In [6]:
rdd_numbers.sum()


1225

In [7]:
rdd_numbers.mean()


25.0

In [8]:
rdd_numbers.count()


49

In [9]:
rdd_numbers.min()


1

In [10]:
rdd_numbers.max()


49

Count how many numbers are even vs. odd.

In [12]:
even_count = rdd_numbers.filter(lambda x: x % 2 == 0).count()
odd_count = rdd_numbers.filter(lambda x: x % 2 != 0).count()
print(f"odd Numbers: {odd_count} even Numbers: {even_count}")

odd Numbers: 25 even Numbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [14]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [15]:
rdd_people.max(key=lambda x: x[1])

('Khaled', 40)

Compute the average age

In [16]:
rdd_people.map(lambda x: x[1]).mean()

31.666666666666668

Group all the names by their age

In [17]:
rdd_people.groupBy(lambda x: x[1]).mapValues(list).collect()

[(30, [('Mona', 30)]),
 (40, [('Khaled', 40)]),
 (25, [('Nada ', 25), ('Nada ', 25)]),
 (35, [('Ahmed', 35), ('Ahmed', 35)])]

Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [22]:
text = "Russia is the largest country in the world by land area\nMoscow is the capital city of Russia\nThe Russian language is one of the most widely spoken languages in the world\nRussia is known for its rich history and culture\nThe Trans-Siberian Railway is the longest railway line in the world\nRussia has a strong tradition in literature, music and ballet\nThe country is famous for its cold winters and vast landscapes\nRussia is a major player in global energy production\n"
with open("russia.txt", "w") as f:
    f.write(text)
rdd_russia = sc.textFile("/content/russia.txt")
rdd_russia.collect()

['Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production']

Count the total number of lines.

In [23]:
rdd_russia.count()

8

Count how many lines contain the word "Russia"

In [24]:
rdd_russia.filter(lambda line: "Russia" in line).count()

6

Find the most 5 frequent word in the file.

In [25]:
words = rdd_russia.flatMap(lambda line: line.split()).map(lambda word: (word.lower(), 1)).reduceByKey(lambda a, b: a + b)
words.takeOrdered(5, key=lambda x: -x[1])

[('the', 10), ('is', 7), ('russia', 5), ('in', 5), ('world', 3)]

Tokenize words

In [26]:
tokens = rdd_russia.flatMap(lambda line: line.split())
print(tokens.collect())

['Russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'Moscow', 'is', 'the', 'capital', 'city', 'of', 'Russia', 'The', 'Russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'Russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'The', 'Trans-Siberian', 'Railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'Russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'The', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of).

In [27]:
stopwords = ['a', 'the', 'is', 'to', 'in', 'of']
filtered_tokens = rdd_russia.flatMap(lambda line: line.split()).filter(lambda word: word.lower() not in stopwords)
print(filtered_tokens.collect())

['Russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'Moscow', 'capital', 'city', 'Russia', 'Russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'Russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'Trans-Siberian', 'Railway', 'longest', 'railway', 'line', 'world', 'Russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [28]:
word_freq = filtered_tokens.map(lambda word: (word.lower(), 1)).reduceByKey(lambda a, b: a + b)
print(word_freq.collect())

[('largest', 1), ('country', 2), ('world', 3), ('by', 1), ('land', 1), ('area', 1), ('capital', 1), ('russian', 1), ('language', 1), ('most', 1), ('widely', 1), ('known', 1), ('for', 2), ('history', 1), ('and', 3), ('line', 1), ('literature,', 1), ('music', 1), ('famous', 1), ('cold', 1), ('winters', 1), ('landscapes', 1), ('player', 1), ('energy', 1), ('production', 1), ('russia', 5), ('moscow', 1), ('city', 1), ('one', 1), ('spoken', 1), ('languages', 1), ('its', 2), ('rich', 1), ('culture', 1), ('trans-siberian', 1), ('railway', 2), ('longest', 1), ('has', 1), ('strong', 1), ('tradition', 1), ('ballet', 1), ('vast', 1), ('major', 1), ('global', 1)]


In [30]:
schema = 'id integer, name string, age integer, salary integer'
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [31]:
df.printSchema()
df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [32]:
df.select("name", "salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [33]:
df.select(fun.avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     6000.0|
+-----------+



Filter employees older than 28

In [34]:
df.filter(df.age > 28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [35]:
df.select("name").distinct().count()

4

Group by a the name column and find average salary

In [36]:
df.groupBy("name").agg(fun.avg("salary").alias("Average")).show()

+------+-------+
|  name|Average|
+------+-------+
|  Omar| 6750.0|
|Mariam| 6750.0|
|   Ali| 4000.0|
|  Sara| 5000.0|
+------+-------+



Find the avg sales

In [ ]:
df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
avg_sales = df1.select(fun.avg("Sales")).collect()[0][0]
print(avg_sales)

400.5


Replace null name with 'Unknown' and sales with the avg sales of the column

In [ ]:
avg_sales = df1.select(fun.avg("Sales")).collect()[0][0]
df1.fillna({"Name": "Unknown", "Sales": avg_sales}).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

